In [ ]:
!pip3 install tensorflow==2.8.0
!pip3 install keras==2.6.0
!pip install matplotlib==3.9.0

  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.8.0 which is incompatible.


  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_instal

In [ ]:
import os

In [ ]:
from keras.datasets import mnist
from keras.layers import Dense, Input, Reshape, Flatten
from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers.advanced_activations import LeakyReLU
# from keras.optimizers_v1 import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
colab_directory = "/content/gan_images2"
os.makedirs(colab_directory, exist_ok=True)

In [ ]:
img_rows = 28
img_columns = 28
channels = 1
img_shape = (img_rows, img_columns, channels)

In [ ]:
#generator
#input is a noise vector and output is a fake (generated ) image
def build_generator():
  noise_shape = (100,) #100*1 1d noise vector as input

  model = Sequential()
  model.add( Dense(256, input_shape=noise_shape))
  model.add( LeakyReLU(alpha=0.2))
  model.add( BatchNormalization(momentum=0.8))
  model.add( Dense(512))
  model.add( LeakyReLU(alpha=0.2))
  model.add( BatchNormalization(momentum=0.8))
  model.add( Dense(1024))
  model.add( LeakyReLU(alpha=0.2))
  model.add( BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

 # model.summary()
  noise = Input(shape=noise_shape)
  img = model(noise) #img stores the output of this model, given the noise vector as the input
  return Model(noise, img)

In [ ]:
yo = build_generator()


In [ ]:
# takes input as an image and outputs the prob of it being a valid(real) image (as predicted by it)
def build_discriminator():

  model = Sequential()
  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(1, activation='sigmoid'))
  # model.summary()

  img = Input(shape=img_shape)
  validity = model(img)

  return Model(img, validity)


In [ ]:
def train(epochs, batch_size=32, save_interval=50):
  norm = 255.0/2
  (X_train, _), (_, _) = mnist.load_data()
  X_train = (X_train.astype(np.float32) - norm)/norm
  X_train = np.expand_dims(X_train, axis = 3) #xtrain has say 50k images each of 28*28, so it is 50k * 28 * 28,
  #we use np. so that each element is now of 3 dimensions, so each is 28*28*1
  half_batch = int(batch_size / 2)

  for epoch in range(epochs):
    ind = np.random.randint(0, X_train.shape[0], half_batch) #random indexes (half_batch jitne) from 0 to len of xtrain - 1 , for explaination say half batch
    #is 50 , so 50 random images choose karne ke liye
    images = X_train[ind] #these 50 images

  #noise vectors generation : each vector ke liye 100 wala , so i need 50 vectors of size 100
    noise = np.random.normal(0, 1, (half_batch, 100)) # one row for each image
    generated_images = generator.predict(noise) #predict 50 random images for these noise vectors

    #now, train discriminator:
    # 50 fake imags with target 0, and 50 real images with target 1
    d_loss_real = discriminator.train_on_batch(images, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((half_batch, 1)))

    d_loss = 0.5*(np.add(d_loss_real, d_loss_fake))

    #now train the generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    valid_output = np.array([1]*batch_size) #here all images so batch_size
    g_loss = combined.train_on_batch(noise, valid_output)

    print("epoch : ", epoch, " d loss avg : ", d_loss, "accuracy % :", 100*d_loss[1], "g loss: ", g_loss) #loss is returned as a tuple : loss, accuracy

    if epoch % save_interval==0:
      save_images(epoch)

In [ ]:
def save_images(epoch):
  r , c= 5, 5
  noise = np.random.normal(0, 1, (r*c, 100)) #will test our generator after some no of epochs, by generating 25 random images
  generated_images = generator.predict(noise) #get the images (generated)
  generated_images = 0.5 * generated_images + 0.5 #scale the images (received from tanh so [-1, 1] so *0.5 -> [-0.5, +0.5] +0.5 -> [0, 1])

  fig, axs = plt.subplots(r, c)
  cnt =0
  for i in range (r):
    for j in range (c):
      axs[i, j].imshow(generated_images[cnt, :, :, 0], cmap='gray')
      axs[i, j].axis('off')
      cnt+=1
  fig.savefig(os.path.join(colab_directory, "mnist_%d.png" % epoch))
  plt.close()

In [ ]:
optimizer = Adam(0.0002, 0.5)
discriminator = build_discriminator()
discriminator.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator()
generator.compile(loss = 'binary_crossentropy', optimizer=optimizer)

z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False

valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss = 'binary_crossentropy', optimizer=optimizer)



In [ ]:
train(epochs =10000,batch_size = 32, save_interval =30)

Streaming output truncated to the last 5000 lines.
epoch :  5000  d loss avg :  [0.59506333 0.78125   ] accuracy % : 78.125 g loss:  0.9697403907775879
epoch :  5001  d loss avg :  [0.66255254 0.53125   ] accuracy % : 53.125 g loss:  0.853410542011261
epoch :  5002  d loss avg :  [0.66517168 0.4375    ] accuracy % : 43.75 g loss:  0.8427916765213013
epoch :  5003  d loss avg :  [0.68846118 0.625     ] accuracy % : 62.5 g loss:  0.8621829152107239
epoch :  5004  d loss avg :  [0.68385556 0.53125   ] accuracy % : 53.125 g loss:  0.985982358455658
epoch :  5005  d loss avg :  [0.60117146 0.65625   ] accuracy % : 65.625 g loss:  0.9743331074714661
epoch :  5006  d loss avg :  [0.71535277 0.46875   ] accuracy % : 46.875 g loss:  0.9525314569473267
epoch :  5007  d loss avg :  [0.59754932 0.71875   ] accuracy % : 71.875 g loss:  0.9846731424331665
epoch :  5008  d loss avg :  [0.75557235 0.5       ] accuracy % : 50.0 g loss:  0.8368869423866272
epoch :  5009  d loss avg :  [0.69104731 0.5937

In [ ]:
generator.save('generator.h5')
discriminator.save('discriminator.h5')